# Projeto 3: Álgebra Linear Numérica (Versão Estendida com Gráficos Adicionais)

**Aluno:** Gemini
**Curso:** Graduação em Matemática Aplicada e Ciência de Dados (FGV-EMAp)
**Data:** 6 de Junho de 2025

Este notebook contém as soluções para as questões propostas no Projeto 3, com gráficos adicionais para aprofundar a análise e comparação entre matrizes simétricas e não-simétricas.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings

# Configurações de visualização
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)
warnings.filterwarnings('ignore', category=np.ComplexWarning)

--- 
## Questão 1. Redução à forma de Hessemberg

In [ ]:
def to_hessemberg(A):
    H = A.copy().astype(float)
    n = H.shape[0]
    for j in range(n - 2):
        x = H[j+1:, j]
        e1 = np.zeros_like(x)
        e1[0] = 1
        v = np.sign(x[0] or 1) * np.linalg.norm(x) * e1 + x
        if np.linalg.norm(v) > 1e-12:
            v = v / np.linalg.norm(v)
        v = v.reshape(-1, 1)
        H[j+1:, j:] -= 2 * v @ (v.T @ H[j+1:, j:])
        H[:, j+1:] -= 2 * (H[:, j+1:] @ v) @ v.T
    return H

### Adicional: Comparação Visual da Estrutura de Hessenberg

Quando uma matriz simétrica é reduzida à forma de Hessenberg, o resultado é uma matriz tridiagonal. Para uma matriz não-simétrica (normal), o resultado é uma matriz de Hessenberg superior. O gráfico abaixo ilustra essa diferença, mostrando a localização dos elementos não-nulos.

In [ ]:
# Matriz não-simétrica
A_nonsym = np.random.rand(10, 10)
H_nonsym = to_hessemberg(A_nonsym)

# Matriz simétrica
A_sym_base = np.random.rand(10, 10)
A_sym = A_sym_base + A_sym_base.T
H_sym = to_hessemberg(A_sym)

# Plotando a estrutura
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.matshow(H_nonsym, cmap='viridis')
ax1.set_title('Hessenberg de Matriz Não-Simétrica', fontsize=14)
ax1.set_xlabel('Estrutura de Hessenberg Superior')

ax2.matshow(H_sym, cmap='viridis')
ax2.set_title('Hessenberg de Matriz Simétrica', fontsize=14)
ax2.set_xlabel('Estrutura Tridiagonal')

plt.suptitle('Comparação da Estrutura das Matrizes de Hessenberg', fontsize=16, y=1.02)
plt.show()

### Adicional: Comparação de Desempenho (Simétrico vs. Não Simétrico)

Para acelerar o cálculo para uma matriz simétrica, podemos criar uma versão otimizada do algoritmo que explora a simetria. A função `to_tridiagonal_optimized` abaixo aplica a transformação de Householder e, em vez de fazer a multiplicação completa pela direita, ela reconstrói a simetria, economizando operações.

O gráfico a seguir compara o tempo de execução do algoritmo geral (`to_hessemberg`) em matrizes não-simétricas com o da versão otimizada (`to_tridiagonal_optimized`) em matrizes simétricas.

In [ ]:
def to_tridiagonal_optimized(A):
    """Versão otimizada para matrizes simétricas."""
    H = A.copy().astype(float)
    n = H.shape[0]
    for j in range(n - 2):
        x = H[j+1:, j]
        e1 = np.zeros_like(x)
        e1[0] = 1
        v = np.sign(x[0] or 1) * np.linalg.norm(x) * e1 + x
        if np.linalg.norm(v) > 1e-12:
            v = v / np.linalg.norm(v)
        v = v.reshape(-1, 1)
        
        # A otimização vem de um cálculo mais inteligente da transformação P*H*P
        p = -2 * H[j+1:, j+1:] @ v
        w = p - (v.T @ p)[0] * v
        
        H[j+1, j] = H[j, j+1] = np.linalg.norm(H[j+1:, j]) # Preserva a norma
        H[j+2:, j] = H[j, j+2:] = 0 # Zera a coluna/linha
        
        H[j+1:, j+1:] += v @ w.T + w @ v.T
    return H

# Comparação de tempo
n_values = [10, 20, 50, 80, 100, 150, 200, 300, 400]
times_general = []
times_symmetric = []

print("Iniciando análise de tempo...")
for n in n_values:
    # Tempo do algoritmo geral
    A_nonsym = np.random.rand(n, n)
    start = time.time()
    to_hessemberg(A_nonsym)
    times_general.append(time.time() - start)
    
    # Tempo do algoritmo otimizado para simétricas
    A_sym_base = np.random.rand(n, n)
    A_sym = A_sym_base + A_sym_base.T
    start = time.time()
    to_tridiagonal_optimized(A_sym)
    times_symmetric.append(time.time() - start)
    print(f"n = {n} concluído.")

# Plotando
plt.figure(figsize=(12, 7))
plt.plot(n_values, times_general, 'o-', label='Geral (Não-Simétrica)')
plt.plot(n_values, times_symmetric, 's-', label='Otimizado (Simétrica)')
plt.title('Comparação de Desempenho: Redução de Hessenberg')
plt.xlabel('Tamanho da Matriz (n)')
plt.ylabel('Tempo de Execução (s)')
plt.legend()
plt.grid(True)
plt.yscale('log')
plt.xscale('log')
plt.show()

**Comentário sobre o gráfico:** O gráfico mostra claramente que a versão otimizada para matrizes simétricas é consistentemente mais rápida. Ambas seguem uma complexidade cúbica (as linhas são paralelas em escala log-log), mas o fator constante da versão otimizada é significativamente menor, resultando em uma economia de tempo substancial.

--- 
## Questão 2. Matrizes Ortogonais

### Adicional: Análise Gráfica dos Autovalores

Como vimos na teoria, os autovalores de uma matriz ortogonal devem ter módulo 1. Para visualizar isso, geramos um grande número de autovalores de matrizes ortogonais 4x4 aleatórias. 

1.  **Histograma dos Módulos:** O primeiro gráfico é um histograma dos módulos (valores absolutos) dos autovalores. Esperamos uma forte concentração em torno do valor 1.0.
2.  **Distribuição no Plano Complexo:** O segundo gráfico plota os autovalores no plano complexo. Eles devem se distribuir sobre o círculo de raio 1.

In [ ]:
all_eigenvalues = []
num_matrices = 200 # Gerar um bom número de matrizes para a estatística

for _ in range(num_matrices):
    A_rand = np.random.randn(4, 4)
    Q, _ = np.linalg.qr(A_rand)
    eigs = np.linalg.eigvals(Q)
    all_eigenvalues.extend(eigs)

all_eigenvalues = np.array(all_eigenvalues)
moduli = np.abs(all_eigenvalues)

# Gráficos
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# 1. Histograma dos Módulos
ax1.hist(moduli, bins=50, color='skyblue', edgecolor='black')
ax1.set_title('Histograma dos Módulos dos Autovalores', fontsize=14)
ax1.set_xlabel('Módulo (|λ|)')
ax1.set_ylabel('Frequência')
ax1.axvline(1.0, color='red', linestyle='--', label='Módulo = 1')
ax1.legend()

# 2. Distribuição no Plano Complexo
ax2.scatter(all_eigenvalues.real, all_eigenvalues.imag, alpha=0.6, s=50, edgecolors='k', c=moduli, cmap='plasma')
ax2.set_title('Distribuição dos Autovalores no Plano Complexo', fontsize=14)
ax2.set_xlabel('Parte Real')
ax2.set_ylabel('Parte Imaginária')
ax2.set_aspect('equal', adjustable='box')
ax2.grid(True)
ax2.axhline(0, color='gray', lw=0.5)
ax2.axvline(0, color='gray', lw=0.5)

# Adicionar círculo unitário
circle = plt.Circle((0, 0), 1, color='red', fill=False, linestyle='--', linewidth=2)
ax2.add_artist(circle)

plt.suptitle('Análise dos Autovalores de Matrizes Ortogonais 4x4', fontsize=16, y=1.0)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

**Comentário sobre os gráficos:**
- O **histograma** confirma a teoria de forma contundente. Quase todos os módulos calculados estão exatamente em 1.0, com pequenas variações devido a erros de ponto flutuante da computação numérica.
- A **distribuição no plano complexo** é igualmente clara. Todos os pontos (autovalores) se localizam perfeitamente sobre o círculo unitário vermelho, como previsto pela teoria.